In [1]:
import os 
import zipfile 
import tensorflow as tf 
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras import layers 
from tensorflow.keras import Model 
import matplotlib.pyplot as plt

In [2]:
# Starting index for the pictures
pic_index = 0


In [12]:
##pip install ultralytics

In [5]:
#for object detection
from ultralytics import YOLO

In [6]:
# create the neural network model
model = YOLO("yolov8m.pt")

In [7]:
#results = model.predict("1.jpg")
#/Users/e123e/Desktop/Data-OD/D-set/data.yaml
results = model.train(data='/Users/e123e/Desktop/Data-OD/D-set/data.yaml', epochs=1, imgsz=640)
#-1-_png.rf.5d9ff973229c8c7fbb33dbefa88141e3.jpg

Ultralytics YOLOv8.1.8 🚀 Python-3.11.5 torch-2.1.2+cpu CPU (Intel Core(TM) i7-9750H 2.60GHz)
engine\trainer: task=detect, mode=train, model=yolov8m.pt, data=/Users/e123e/Desktop/Data-OD/D-set/data.yaml, epochs=1, time=None, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True